## Mask Extraction

This notebook illustrates an example to extract segmentation geometries (polygons) in WKT format from a grayscale image mask.

This example uses:  
- `PIL` library for image I/O  
- `imatics` to extract mask geometry coordinates 
- `shapely` for post-processing extracted polygons and exporting into WKT format

You can dump extracted mask segmentations as a json file. 
Extracted WKTs are visualized on the base image to verify successful extraction at the end.
The WKTs generated from this script will be formatted with absolute coordinates. To use
these geometries with Centaur products you must first convert them to relative coordinates.
Check out the coordinate conversion recpie (@TODO: link recipe) to perform the conversion.

In [ ]:
# install dependencies
! pip install -r requirements.txt

In [ ]:
# imports
import numpy as np
from PIL import Image

import imantics
import shapely.wkt
import shapely.geometry
import shapely.validation

In [ ]:
BASE_IMAGE_FILE_PATH = './example/input/example_base.png'
MASK_IMAGE_FILE_PATH = './example/input/example_mask.png'

JSON_OUTPUT_PATH = './example/output/example_mask_wkt.json'

### extract wkt segmentations from mask_image

In [ ]:
# load image
mask_image = Image.open(MASK_IMAGE_FILE_PATH)
# extract pixel_array
pixel_array = np.asarray(mask_image)
# identify mask pixels
distinct_pixels_values = np.unique(pixel_array) 
mask_pixels = distinct_pixels_values[distinct_pixels_values.nonzero()]

mask_segmentations = []
for focus_mask_pixel in mask_pixels:
    # isolate mask pixel
    pixel_array_copy = pixel_array.copy()
    pixel_array_copy[pixel_array_copy != focus_mask_pixel] = 0

    # extract mask
    mask_polygon = imantics.Mask(pixel_array_copy).polygons().points[0]
    # post-process masks into valid wkts
    mask_polygon = shapely.geometry.Polygon(mask_polygon)
    valid_mask_polygon = shapely.validation.make_valid(mask_polygon)
    simplified_valid_mask_polygon = valid_mask_polygon.simplify(tolerance=2, preserve_topology=True)
    focus_mask_wkt = shapely.wkt.dumps(simplified_valid_mask_polygon, trim=True)

    # add extracted masks to json object
    focus_mask_dict = {"mask_pixel": focus_mask_pixel.item(), "mask_wkt": focus_mask_wkt}
    mask_segmentations.append(focus_mask_dict)

mask_segmentations

In [ ]:
# dump extracted mask segmentations as json
import json
with open(JSON_OUTPUT_PATH, "w", encoding="utf-8") as fp:
    json.dump(mask_segmentations, fp, ensure_ascii=False, indent=4)

### visualize extracted wkts on base image

In [ ]:
from PIL import ImageDraw, ImageFont
from matplotlib import cm, font_manager

# load base image
base_image = Image.open(BASE_IMAGE_FILE_PATH)
cmap = cm.get_cmap('hsv')
font = font_manager.FontProperties(family='sans-serif', weight='bold')
font = ImageFont.truetype(font_manager.findfont(font), 48)

# overlay mask
mask_overlay = base_image.copy()
for mask in mask_segmentations:
    poly_coords = list(shapely.wkt.loads(mask['mask_wkt']).exterior.coords)
    ImageDraw.Draw(mask_overlay).polygon(poly_coords, outline = cmap(mask["mask_pixel"], bytes=True), width=2)

# show overlay
ImageDraw.Draw(base_image).text((100, 100), "Base Image", (255,255,255), font=font)
ImageDraw.Draw(mask_image := mask_image.convert("RGBA")).text((100, 100), "Mask Image", (255,255,255), font=font)
ImageDraw.Draw(mask_overlay).text((100, 100), "Mask Overlay", (255,255,255), font=font)
overlay_strip = Image.fromarray(np.hstack((np.array(base_image), 
                                           np.array(mask_image), 
                                           np.array(mask_overlay))))
overlay_strip